In this cell, please type your name and SID

# ECON 140 - Problem Set 8

## INSTRUCTIONS

* Please step through this problem set, produce or fix the code, and run it to produce output. If provided code produces an error, you should try to correct it.

## Modeling discrete outcome

In this tutorial, we will look at several ways to model discrete outcome using [generalized linear models (GLM)](https://en.wikipedia.org/wiki/Generalized_linear_model), of which OLS is a special case. Generalized linear models typically make use of a class of methods called [maximum likelihood estimation (MLE)](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation), which yields the OLS estimator if the OLS model assumes normality in the error term (though please recall that normality is not required for OLS).

To get started, copy and paste the following code into the code block below and run it.
```R
packlist = c("wooldridge", "MASS")
install.packages(packlist[!(packlist %in% installed.packages()[, 1])])
```

### Binary-response model with [logistic regression (logit)](https://en.wikipedia.org/wiki/Logistic_regression)

Binary response refers to an outcome of whether something occurs or does not occur. There are quite a few ways to model binary response, but logistic regression is the most common. We will compare the performance of OLS and logit for fitting binary response data using the `affairs` dataset from the `wooldridge` package.

Run the following snippet in the code block below:
```R
data("affairs", package = "wooldridge")
head(affairs)
```


A description of the dataset can be found in the `wooldridge` documntation file [here](https://cran.r-project.org/web/packages/wooldridge/wooldridge.pdf). We want to model the relationship between the choice to have an affair (or not) and some predictors. Among our choices, we now look at a comparison between OLS and logit. Run the following snippet in the code block below:
```R
data("affairs", package = "wooldridge")
m1.ols <- lm(affair ~ male + age + yrsmarr + kids + relig + educ +
               factor(occup) + ratemarr,
               data = affairs)
m1.logit <- glm(affair ~ male + age + yrsmarr + kids + relig + educ +
                  factor(occup) + ratemarr,
                  family = "binomial",
                  data = affairs)
```

Produce the summary of both models in the code block below using the `summary()` function:

Run the following snippet in the code block below to produce histograms of the fitted values of each model:
```R
par(mfrow = c(1, 2)) #creates a 1x2 grid which subsequent graphs populate
hist(m1.ols$fitted.values)
hist(m1.logit$fitted.values)
```

As you can see, OLS produces fitted values that can be negative. This is at odds with the percentage/probability interpretation of the fitted and predicted values based on this model. Logit, on the other hand, produces fitted values strictly within the support of [0, 1] according to the logistic function
$$ \hat{y} = (1 + e^{-X'\hat{\Beta}})^{-1} $$

You can test the above with the following code
```R
sum(m1.logit$fitted.values - 1 / (1 + exp( - predict(m1.logit))))
```

which should be to 0 or close to 0 (due to rounding). `predict()` used with a `glm()` output produces the value of the linear combination $X'\hat{B}$ only and the link function (in this case logit) needs to be applied to obtain the fitted probability value.

Because the outcome variable is binary and discrete, we want to make predictions that are the same. One of the ways to do so is to convert the fitted value into binary response at 50% probability cutoff. Run the following snippet in the code block below to produce these predictions
```R
m1.ols.predict <- ifelse(m1.ols$fitted.values >= .5, 1, 0)
m1.logit.predict <- ifelse(m1.logit$fitted.values >= .5, 1, 0)
```

The above code are effectively the same, but not exactly the same, as rounding to the nearest integer, as for the OLS model it has the potential of rounding to less than 0 or more than one. With this prediction concept, we can count the number of times the models predict correctly to obtain a percentage of correct predictions. Run the snippet below to see:
```R
list(Name = "Percentage Correct",
     OLS = sum(m1.ols.predict == affairs$affair) / dim(affairs)[1],
     Logit = sum(m1.logit.predict == affairs$affair) / dim(affairs)[1])
```

Alternatively, we can use root mean squared error (RMSE), another common measure for predictive accuracy and model fit. First define the RMSE function by running the following snippet:
```R
rmse <- function(x, y){
  if(is.vector(x) & is.vector(y) & length(x) == length(y)){
    sqrt(sum((x - y) ^ 2) / length(x))
  } else {
    message("Inputs must be vectors of the same length.")
  }
```

Applying the `rmse()` function to the fitted values:
```R
list(Name = "RMSE",
     OLS = rmse(m1.ols$fitted.values, affairs$affair),
     Logit = rmse(m1.logit$fitted.values, affairs$affair))
```

In this example, you should see that logit performs slightly better than OLS in both percentage correct. It is important to note, however, that these are not the only measures of fit, accuracy, and appropriateness of a method. OLS often, but not always, has an advantage in producing smaller RSME due to RMSE being directly related to "least squares" (the "LS" in OLS), but may not be the appropriate model. This will be apparent in the subsequent examples.

### Poisson regression and count data

`naffairs`, or the number of extramarital affairs in the past year from this data set, can be used as an alternative outcome variable. As a _count variable_, which takes on non-negative integer values, it can be modeled using [Poisson regression](https://en.wikipedia.org/wiki/Poisson_regression), from [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution). Poisson regression is suitable when the conditional distribution of the outcome variable does not show dispersion, _ie._ when $\sigma^2 = \mu = \lambda$. Run the snippets below to fit `naffairs` to covariates with OLS and Poisson regression:
```R
m2.ols <- lm(naffairs ~ male + age + yrsmarr + kids + relig + educ +
               factor(occup) + ratemarr,
               data = affairs)
m2.poisson <- glm(naffairs ~ male + age + yrsmarr + kids + relig + educ +
                    factor(occup) + ratemarr,
                    family = "poisson",
                    data = affairs)
```

Use the `summary()` function in the code block below to show the regression results of the two models

While there can be several predicted value concepts, we'll keep things simple in this example and take the fitted values rounded down to integers as the prediction for the discrete count. Run the following snippet in the code block below:
```R
m2.ols.predict <- floor(m2.ols$fitted.values)
m2.poisson.predict <- floor(floor(m2.poisson$fitted.values))
# with glm(..., family = Poisson), model$fitted.values is equivalent to
# exp(predict(model))
```

Now compare the predicted response sets to the actual response:
```R
par(mfrow = c(1, 3))
hist(affairs$naffairs)
hist(m2.ols.predict)
hist(m2.poisson.predict)
```

As you can see, the distribution of Poisson regression predictions is much closer to that of the actual distribution of responses, whereas OLS predictions show strong central tendency that does not comport with the actual data. To get a rough measure the percentage correctly predicted (which may not be a suitable measure in this simple form):
```R
list(Name = "Percentage Correct",
     OLS = sum(m2.ols.predict == affairs$naffairs) /
       dim(affairs)[1],
     Poisson = sum(m2.poisson.predict == affairs$naffairs) /
       dim(affairs)[1])
```

In this crude measure, Poisson yields much better result than OLS.

In the code block below, use the `list()` function like the examples above to compare the RMSE of the two models

### Ordered logit and ranked response

The third and last type of discrete outcome we will touch upon in this tutorial is ranked response, such as restaurant review (1 to 5 stars), competitive ranking (gold, silver, bronze), discrete conditions (good, okay, bad), _etc._ Conceivably, these can also be modeled with OLS, but are much better modeled using [ordered logit](https://en.wikipedia.org/wiki/Ordered_logit). In this example, we will use the `polr()` function from the `MASS` package to produce ordered logit models. To get started, run the following snippet in the code block below:
```R
library(MASS)

m3.ols <- lm(ratemarr ~ male + age + yrsmarr + kids + relig + educ +
               factor(occup),
             data = affairs)

m3.ologit <- polr(as.factor(ratemarr) ~ male + age + yrsmarr + kids + relig +
                    educ + factor(occup),
                  data = affairs)
```

Note that `ratemarr` must be converted to factors in the `polr()` formula to distinguish itself as a discrete variable. This is not done in OLS because we treat `ratemarr` as cardinal numbers there.

Use the `summary()` function in the code block below to show the regression results of the two models

To produce predicted response from the OLS model, we'll simply round the fitted values to the closest integer:
```R
m3.ols.predict <- round(m3.ols$fitted.values)
```

For the ordered logit model, let's first look at the fitted value, which is not a vector, but a table:
```R
head(m3.ologit$fitted.values)
```

If you look at it closely, you will see that each row sums up to 1. To check, run the following snippet:
```R
rowSums(head(m3.ologit$fitted.values))
```

Basically, for each of the ratings, the model produces a probability of the respondent choosing that rating based on the covariates. To get the predicted response, we may use the rating that is the most likely. Unlike the `glm()` function, `predict()` used with `polr()` produces the predicted response directly.
```R
m3.ologit.predict <- as.numeric(predict(m3.ologit))
```

Note `as.numeric()` coerces the ranked predictions into numbers that we can use for the following analysis. The above is equivalent to picking the column number of the highest number of each row. To check
```R
sum(apply(m3.ologit$fitted.values, 1, which.max) - m3.ologit.predict)
```

Should be exactly 0.

Now let's plot the distribution of the actual response alongside the predictions:
```R
par(mfrow = c(1, 3))
hist(affairs$ratemarr)
hist(m3.ols.predict)
hist(m3.ologit.predict)
```

Like with Poisson regression, the ordered logit produces predictions that are much closer in distribution than OLS, which once again exhibits central tendency. We will use the same crude measure of accuracy here to compare the spot-on rates between OLS and ordered logit:
```R
list(Name = "Percentage Correct",
     OLS = sum(m3.ols.predict == affairs$ratemarr) / dim(affairs)[1],
     "Ordered Logit" = sum(m3.ologit.predict == affairs$ratemarr) / dim(affairs)[1])
)
```


In the code block below, use the `list()` function like the examples above to compare the RMSE of the two models

In the text block below, comment on what you observe with the RMSE between OLS and ordered logit.

### Additional readings
UCLA has some neat tutorials on these topics using R that you should check out:
* [Binary logit](https://stats.oarc.ucla.edu/r/dae/logit-regression/)
* [Ordered logit](https://stats.oarc.ucla.edu/r/dae/ordinal-logistic-regression/)
* [Poisson regression](https://stats.oarc.ucla.edu/r/dae/poisson-regression/)
* [Multinomial logit](https://stats.oarc.ucla.edu/r/dae/multinomial-logistic-regression/), which is used to model choice from more than two possible options that do not fit on a clear scale, _e.g._ which brand of cereal you buy, which major you choose, _etc_. `mlogit`([documentation](https://cran.r-project.org/web/packages/mlogit/index.html)) and `nnet`("neural network", [documentation](https://cran.r-project.org/web/packages/nnet/index.html)) packages for `R` can be used for this class of models.